# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()

# 连接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [4]:
def get_data(now_date):
    
    for index, date in enumerate(now_date):
        # df1 = None
        sql=f'''
        SELECT o.create_time, o.order_number,o.channel,c.`name`,o.merchant_name,o.`status`,o.cancel_reason,JSON_EXTRACT(ori.decision_result, "$.rejected") as rejected,JSON_EXTRACT(ori.decision_result, "$.status") as risk_status,order_method , tbc.classify_id
        from db_digua_business.t_order as o
        left join db_digua_business.t_order_risk as ori on o.id = ori.order_id  
        left join db_digua_business.t_order_details as od on o.id = od.order_id
        left join db_digua_business.t_platform_activity as pa on pa.id = o.activity_id
        left join db_digua_business.t_channel as c on c.scene = o.channel 
        left join db_digua_business.t_brand_classify tbc on tbc.id = od.product_id
        where `day` >=90 AND pa.type = 2
        and o.create_time >= date_add('{date}', interval -14 day)
        and o.create_time <= date_add('{date}', interval -7 day)
        and o.`status` not in (1,13)
        '''
        df = query(sql)
        df.insert(10, '统计时间', date)
        if index==0:
            df1=df.copy()
        else:
            df1 = pd.concat([df1, df], ignore_index=True)
            
    return df1
df = get_data(['2024-11-17','2024-11-18','2024-11-19','2024-11-20','2024-11-21','2024-11-22','2024-11-23','2024-11-24','2024-11-25','2024-11-26','2024-11-27', '2024-11-28'])
df

,create_time,order_number,channel,name,merchant_name,status,cancel_reason,rejected,risk_status,order_method,统计时间,classify_id
0,2024-11-03 00:23:45,A2024110300234431,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,迪瓜优选,10,None,"[""命中策略strategy_240801强拒""]",1,1,2024-11-17,None
1,2024-11-03 00:26:10,A2024110300261033,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,迪瓜优选,10,None,"[""命中策略strategy_240801强拒""]",1,1,2024-11-17,None
2,2024-11-03 00:49:56,A2024110300495680,3802fd5a-7389-4870-9900-1542879fd5ba,支付宝直播,维客自营账户,10,"88888_林婉婷审核订单A2024110300495680，审核结果：不通过,msg=客户...",[],0,0,2024-11-17,None
3,2024-11-03 01:00:35,A202411030100341,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,迪瓜优选,10,None,"[""命中自有模型强拒""]",1,1,2024-11-17,None
4,2024-11-03 01:10:19,A202411030110198,9c956f548ccf415eb59b314fa4d3fe69,芝麻信用租物夏促,维客自营账户,10,系统操作审核订单A202411030110198，出库前风控审核结果：不通过,"[""命中策略strategy_240927强拒""]",1,0,2024-11-17,None
...,...,...,...,...,...,...,...,...,...,...,...,...
22347,2024-11-20 23:27:15,A2024112023271549,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,维客自营账户,10,"88888_李巧凤审核订单A2024112023271549，审核结果：不通过,msg=客户...",[],0,1,2024-11-28,None
22348,2024-11-20 23:33:04,A2024112023330356,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,迪瓜优选,10,None,"[""命中策略strategy_240801强拒""]",1,1,2024-11-28,None
22349,2024-11-20 23:44:47,A2024112023444773,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,维客自营账户,10,系统操作审核订单A2024112023444773，出库前风控审核结果：不通过,"[""命中极信sc32007分强拒""]",1,1,2024-11-28,None
22350,2024-11-20 23:48:33,A2024112023483382,83c21e3c23fb4687baa2fb8cdca4f281,芝麻租物,迪瓜优选,10,None,"[""命中非免押用户拒绝""]",1,1,2024-11-28,None


In [5]:
sql2 = '''
SELECT o.create_time, o.order_number,o.channel,c.`name`,o.merchant_name,o.`status`,o.cancel_reason,JSON_EXTRACT(ori.decision_result, "$.rejected") as rejected,JSON_EXTRACT(ori.decision_result, "$.status") as risk_status,order_method , tbc.classify_id
from db_digua_business.t_order as o
        left join db_digua_business.t_order_risk as ori on o.id = ori.order_id  
        left join db_digua_business.t_order_details as od on o.id = od.order_id
        left join db_digua_business.t_platform_activity as pa on pa.id = o.activity_id
        left join db_digua_business.t_channel as c on c.scene = o.channel 
        left join db_digua_business.t_brand_classify tbc on tbc.id = od.product_id
        where `day` >=90 AND pa.type = 2
        and o.create_time >= '2024-11-02 00:00:00'
        and o.create_time <= '2024-11-21 00:00:00'
        and o.`status` not in (1,13)
'''
df2 = query(sql2)

# 处理数据

In [6]:
# 填充risk_status字段，商家的订单填充为0
# df['risk_status'] = df['risk_status'].fillna('2')
# 获取状态
def df_status(df):
    df['机审强拒'] = np.where(df['risk_status']=='1', 1, 0)
    df['机审通过件'] = np.where(df['risk_status']=='0', 1, 0)
    df['人审拒绝'] = np.where((df['risk_status']!='1')&(df['cancel_reason'].str.contains('审核不通过：')), 1, 0)
    df['风控通过件'] = np.where((df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)
    df['客户取消'] = np.where(df['cancel_reason'].str.contains('客户申请取消：'), 1, 0)
    df['无法联系'] = np.where(df['cancel_reason'].str.contains('用户无法联系：'), 1, 0)
    df['出库前风控强拒'] = np.where(df['cancel_reason'].str.contains('出库前风控'), 1, 0)
    df['待审核'] = np.where(df['status']==12, 1, 0)
    df['出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df["status"].isin([2,3,4,5,6,8,15])), 1, 0)
    return df
df = df_status(df)
df2 = df_status(df2)
df_zw = df[df['name'].str.contains(r'芝麻|租物')]


In [7]:
def data_group(df, model):
    df['统计周期'] = pd.to_datetime(df[model]) - pd.Timedelta(days=14)
    df_group = df.groupby(model).agg({'order_number': 'count', '机审强拒': 'sum', '机审通过件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '出库': 'sum'})
    df_group.rename(columns={'order_number': '进件数'}, inplace=True)
    df_group["强拒比例"] = df_group["机审强拒"] / df_group["进件数"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x: format(x, ".2%"))
    df_group["人审拒绝率"] = df_group["人审拒绝"] / df_group["进件数"]
    df_group["人审拒绝率"] = df_group["人审拒绝率"].apply(lambda x: format(x, ".2%"))
    df_group["风控通过率"] = df_group["风控通过件"] / df_group["进件数"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x: format(x, ".2%"))
    df_group["取消率"] = df_group["客户取消"] / df_group["进件数"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x: format(x, ".2%"))
    
    df_group["无法联系占比"] = df_group["无法联系"] / df_group["进件数"]
    df_group["无法联系占比"] = df_group["无法联系占比"].apply(lambda x: format(x, ".2%"))
    df_group["出库前强拒比例"] = df_group["出库前风控强拒"] / df_group["进件数"]
    df_group["出库前强拒比例"] = df_group["出库前强拒比例"].apply(lambda x: format(x, ".2%"))
    
    df_group["进件出库率"] = df_group["出库"] / df_group["进件数"]
    df_group["进件出库率"] = df_group["进件出库率"].apply(lambda x: format(x, ".2%"))
    
    
    return df_group

# 快捷下单

In [9]:
df_1 = df_zw[df_zw['order_method']==1]
df_1_group = data_group(df_1, '统计时间')
df_1_group_new = df_1_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl') as writer:
#     df_1_group_new.to_excel(writer, sheet_name='快捷下单')
df_1_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,1151,723,62.81%,380,65,5.65%,363,31.54%,146,12.68%,11,0.96%,32,2.78%,0,171,14.86%
2024-11-18,1182,730,61.76%,398,70,5.92%,382,32.32%,142,12.01%,13,1.10%,36,3.05%,0,177,14.97%
2024-11-19,1197,750,62.66%,396,61,5.10%,386,32.25%,149,12.45%,14,1.17%,34,2.84%,0,169,14.12%
2024-11-20,1208,752,62.25%,407,61,5.05%,395,32.70%,154,12.75%,14,1.16%,39,3.23%,0,171,14.16%
2024-11-21,1225,762,62.20%,412,54,4.41%,409,33.39%,159,12.98%,15,1.22%,43,3.51%,0,176,14.37%
2024-11-22,1172,716,61.09%,402,48,4.10%,408,34.81%,154,13.14%,15,1.28%,44,3.75%,0,173,14.76%
2024-11-23,1182,711,60.15%,404,44,3.72%,427,36.13%,161,13.62%,12,1.02%,41,3.47%,0,183,15.48%
2024-11-24,1159,695,59.97%,394,38,3.28%,426,36.76%,166,14.32%,11,0.95%,43,3.71%,0,183,15.79%
2024-11-25,1127,672,59.63%,369,32,2.84%,423,37.53%,172,15.26%,13,1.15%,40,3.55%,0,177,15.71%


# 非快捷下单

In [10]:
df_0 = df_zw[df_zw['order_method']==0]
df_0_group = data_group(df_0, '统计时间')
df_0_group_new = df_0_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_0_group_new.to_excel(writer, sheet_name='非快捷下单')
df_0_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,343,167,48.69%,83,24,7.00%,152,44.31%,39,11.37%,3,0.87%,9,2.62%,0,96,27.99%
2024-11-18,372,184,49.46%,87,23,6.18%,165,44.35%,37,9.95%,3,0.81%,8,2.15%,0,110,29.57%
2024-11-19,388,199,51.29%,93,17,4.38%,172,44.33%,40,10.31%,3,0.77%,9,2.32%,0,107,27.58%
2024-11-20,393,192,48.85%,99,19,4.83%,182,46.31%,43,10.94%,3,0.76%,7,1.78%,0,110,27.99%
2024-11-21,401,187,46.63%,107,18,4.49%,196,48.88%,49,12.22%,2,0.50%,7,1.75%,0,117,29.18%
2024-11-22,393,179,45.55%,109,13,3.31%,201,51.15%,52,13.23%,1,0.25%,8,2.04%,0,121,30.79%
2024-11-23,388,183,47.16%,109,13,3.35%,192,49.48%,54,13.92%,1,0.26%,6,1.55%,0,111,28.61%
2024-11-24,359,173,48.19%,99,14,3.90%,172,47.91%,47,13.09%,1,0.28%,7,1.95%,0,101,28.13%
2024-11-25,330,155,46.97%,93,17,5.15%,158,47.88%,43,13.03%,1,0.30%,6,1.82%,0,92,27.88%


# 自营

In [11]:
df_zy = df_zw[df_zw['risk_status'].notna()]
df_zy_group = data_group(df_zy, '统计时间')
df_zy_group_new = df_zy_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zy_group_new.to_excel(writer, sheet_name='自营')
df_zy_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,1353,890,65.78%,463,65,4.80%,398,29.42%,176,13.01%,11,0.81%,41,3.03%,0,189,13.97%
2024-11-18,1399,914,65.33%,485,69,4.93%,416,29.74%,170,12.15%,12,0.86%,44,3.15%,0,199,14.22%
2024-11-19,1438,949,65.99%,489,57,3.96%,432,30.04%,179,12.45%,12,0.83%,43,2.99%,0,195,13.56%
2024-11-20,1450,944,65.10%,506,56,3.86%,450,31.03%,184,12.69%,12,0.83%,46,3.17%,0,201,13.86%
2024-11-21,1468,949,64.65%,519,47,3.20%,472,32.15%,195,13.28%,13,0.89%,50,3.41%,0,208,14.17%
2024-11-22,1406,895,63.66%,511,39,2.77%,472,33.57%,192,13.66%,11,0.78%,52,3.70%,0,208,14.79%
2024-11-23,1407,894,63.54%,513,31,2.20%,482,34.26%,204,14.50%,9,0.64%,47,3.34%,0,211,15.00%
2024-11-24,1361,868,63.78%,493,29,2.13%,464,34.09%,203,14.92%,8,0.59%,50,3.67%,0,201,14.77%
2024-11-25,1289,827,64.16%,462,25,1.94%,437,33.90%,202,15.67%,9,0.70%,46,3.57%,0,184,14.27%


# 自营快捷下单

In [12]:
df_zy_1 = df_zy[df_zy['order_method']==1]
df_zy_1_group = data_group(df_zy_1, '统计时间')
df_zy_1_group_new = df_zy_1_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zy_1_group_new.to_excel(writer, sheet_name='自营快捷下单')
df_zy_1_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,1103,723,65.55%,380,50,4.53%,330,29.92%,141,12.78%,9,0.82%,32,2.90%,0,168,15.23%
2024-11-18,1128,730,64.72%,398,53,4.70%,345,30.59%,137,12.15%,10,0.89%,36,3.19%,0,174,15.43%
2024-11-19,1146,750,65.45%,396,45,3.93%,351,30.63%,144,12.57%,10,0.87%,34,2.97%,0,167,14.57%
2024-11-20,1159,752,64.88%,407,44,3.80%,363,31.32%,147,12.68%,10,0.86%,39,3.36%,0,169,14.58%
2024-11-21,1174,762,64.91%,412,37,3.15%,375,31.94%,151,12.86%,11,0.94%,43,3.66%,0,174,14.82%
2024-11-22,1118,716,64.04%,402,31,2.77%,371,33.18%,146,13.06%,10,0.89%,44,3.94%,0,170,15.21%
2024-11-23,1115,711,63.77%,404,24,2.15%,380,34.08%,153,13.72%,8,0.72%,41,3.68%,0,175,15.70%
2024-11-24,1089,695,63.82%,394,21,1.93%,373,34.25%,159,14.60%,7,0.64%,43,3.95%,0,166,15.24%
2024-11-25,1041,672,64.55%,369,16,1.54%,353,33.91%,162,15.56%,8,0.77%,40,3.84%,0,151,14.51%


# 自营非快捷下单

In [13]:
df_zy_0 = df_zy[df_zy['order_method']==0]
df_zy_0_group = data_group(df_zy_0, '统计时间')
df_zy_0_group_new = df_zy_0_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_zy_0_group_new.to_excel(writer, sheet_name='自营非快捷下单')
df_zy_0_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,250,167,66.80%,83,15,6.00%,68,27.20%,35,14.00%,2,0.80%,9,3.60%,0,21,8.40%
2024-11-18,271,184,67.90%,87,16,5.90%,71,26.20%,33,12.18%,2,0.74%,8,2.95%,0,25,9.23%
2024-11-19,292,199,68.15%,93,12,4.11%,81,27.74%,35,11.99%,2,0.68%,9,3.08%,0,28,9.59%
2024-11-20,291,192,65.98%,99,12,4.12%,87,29.90%,37,12.71%,2,0.69%,7,2.41%,0,32,11.00%
2024-11-21,294,187,63.61%,107,10,3.40%,97,32.99%,44,14.97%,2,0.68%,7,2.38%,0,34,11.56%
2024-11-22,288,179,62.15%,109,8,2.78%,101,35.07%,46,15.97%,1,0.35%,8,2.78%,0,38,13.19%
2024-11-23,292,183,62.67%,109,7,2.40%,102,34.93%,51,17.47%,1,0.34%,6,2.05%,0,36,12.33%
2024-11-24,272,173,63.60%,99,8,2.94%,91,33.46%,44,16.18%,1,0.37%,7,2.57%,0,35,12.87%
2024-11-25,248,155,62.50%,93,9,3.63%,84,33.87%,40,16.13%,1,0.40%,6,2.42%,0,33,13.31%


# 商家

In [14]:
df_sj = df_zw[df_zw['risk_status'].isna()]
df_sj_group = data_group(df_sj, '统计时间')
df_sj_group_new = df_sj_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
# with pd.ExcelWriter('F:/需求/瑞彤需求/7天统计周期数据转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_sj_group_new.to_excel(writer, sheet_name='商家')
df_sj_group_new

,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,出库,进件出库率
统计时间,,,,,,,,,,,,,,,,,
2024-11-17,141,0,0.00%,0,24,17.02%,117,82.98%,9,6.38%,3,2.13%,0,0.00%,0,78,55.32%
2024-11-18,155,0,0.00%,0,24,15.48%,131,84.52%,9,5.81%,4,2.58%,0,0.00%,0,88,56.77%
2024-11-19,147,0,0.00%,0,21,14.29%,126,85.71%,10,6.80%,5,3.40%,0,0.00%,0,81,55.10%
2024-11-20,151,0,0.00%,0,24,15.89%,127,84.11%,13,8.61%,5,3.31%,0,0.00%,0,80,52.98%
2024-11-21,158,0,0.00%,0,25,15.82%,133,84.18%,13,8.23%,4,2.53%,0,0.00%,0,85,53.80%
2024-11-22,159,0,0.00%,0,22,13.84%,137,86.16%,14,8.81%,5,3.14%,0,0.00%,0,86,54.09%
2024-11-23,163,0,0.00%,0,26,15.95%,137,84.05%,11,6.75%,4,2.45%,0,0.00%,0,83,50.92%
2024-11-24,157,0,0.00%,0,23,14.65%,134,85.35%,10,6.37%,4,2.55%,0,0.00%,0,83,52.87%
2024-11-25,168,0,0.00%,0,24,14.29%,144,85.71%,13,7.74%,5,2.98%,0,0.00%,0,85,50.60%


# 商家快捷下单

In [15]:
df2['统计时间'] = df2.create_time.dt.date

In [16]:
df_sj2_kj = df2[(df2.name.str.contains(r'芝麻|租物'))&(df2.risk_status.isna())&(df2.order_method==1)&(df2.merchant_name.isin(['优优2店', '乙辉数码']))]
# df_sj2_kj
df_sj_kj_group = data_group(df_sj2_kj, '统计时间')
df_sj_kj_group_new = df_sj_kj_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
df_sj_kj_group_new[['进件数', '人审拒绝','风控通过件', '客户取消', '无法联系', '出库前风控强拒',  '出库']].sum()

进件数        63
人审拒绝       26
风控通过件      37
客户取消        3
无法联系        2
出库前风控强拒     0
出库          0
dtype: int64

In [17]:
df_sj_1 = df_sj[df_sj['order_method']==1]
df_sj_1_group = data_group(df_sj_1, '统计时间')
df_sj_1_group_new = df_sj_1_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
df_sj_1_group_new[['进件数', '人审拒绝','风控通过件', '客户取消', '无法联系', '出库前风控强拒',  '出库']].sum()

进件数        830
人审拒绝       194
风控通过件      636
客户取消        94
无法联系        44
出库前风控强拒      0
出库         194
dtype: int64

# 商家非快捷下单

In [18]:
df_sj2_fkj = df2[(df2.name.str.contains(r'芝麻|租物'))&(df2.risk_status.isna())&(df2.order_method==0)&(df2.merchant_name.isin(['优优2店', '乙辉数码']))]
df_sj2_fkj_group = data_group(df_sj2_fkj, '统计时间')
df_sj2_fkj_group_new = df_sj2_fkj_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
df_sj2_fkj_group_new[['进件数', '人审拒绝','风控通过件', '客户取消', '无法联系', '出库前风控强拒',  '出库']].sum()

进件数        12
人审拒绝        7
风控通过件       5
客户取消        0
无法联系        1
出库前风控强拒     0
出库          1
dtype: int64

In [19]:
df_sj_0 = df_sj[df_sj['order_method']==0]
df_sj_0_group = data_group(df_sj_0, '统计时间')
df_sj_0_group_new = df_sj_0_group[['进件数','机审强拒','强拒比例','机审通过件','人审拒绝','人审拒绝率','风控通过件','风控通过率','客户取消','取消率','无法联系','无法联系占比','出库前风控强拒','出库前强拒比例','待审核','出库','进件出库率']]
df_sj_0_group_new[['进件数', '人审拒绝','风控通过件', '客户取消', '无法联系', '出库前风控强拒',  '出库']].sum()

进件数        1030
人审拒绝         77
风控通过件       953
客户取消         43
无法联系          5
出库前风控强拒       0
出库          804
dtype: int64